<a href="https://colab.research.google.com/github/tirtha0809/Machine-Learning/blob/main/REMOVE_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras as kf
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot
from os import makedirs
from numpy import dstack
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score,roc_auc_score,roc_curve,auc
from statistics import mean
import time

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Importing DataSet

In [6]:
data = pd.read_csv('/content/drive/MyDrive/LANDSLIDE/LandslideData .csv')
dataset = data.dropna(axis = 0, how ='any')
X =  dataset.iloc[:,1:-1].values
y = dataset.iloc[:, -1].values
print(len(dataset))

3991


In [7]:
#smote = SMOTE()

In [8]:
#x_smote, y_smote = smote.fit_resample(X, y)
smote= SMOTE(k_neighbors = 3)
x_smote,y_smote = smote.fit_resample(X,y)

In [9]:
from keras.models import load_model

In [10]:
# create a custom function to load model
def load_all_models(n_models):
    all_models = list()
    for i in range(n_models):
        # filename
        filename = 'north/model' + str(i + 1) + '.h5'
        # load model
        model = load_model(filename)
        # Add a list of all the weaker learners
        all_models.append(model)
        #print('>loaded %s' % filename)
    return all_models

## Stacking models

In [11]:
# create stacked model input dataset as outputs from the ensemble
def stacked_dataset(members, inputX):
    stackX = None
    for model in members:
        # make prediction
        yhat = model.predict(inputX, verbose=0)
        # stack predictions into [rows, members, probabilities]
        if stackX is None:
            stackX = yhat #
        else:
            stackX = dstack((stackX, yhat))
    # flatten predictions to [rows, members x probabilities]
    stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
    return stackX

## Fit meta-learner

In [12]:
# fit a model based on the outputs from the ensemble members
def fit_stacked_model(members, inputX, inputy):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX)
    #print("Models Test time: ",(end-start) * 10**3, "ms")
    # fit the meta learner
    # model = LogisticRegression() #meta learner
    # model = SVC(kernel = 'rbf',random_state = 0,probability= True)
    #model = SVC(kernel = 'linear', random_state = 0,probability=True)
    # model = GaussianNB()
    #model = KNeighborsClassifier(n_neighbors = 3, metric = 'minkowski', p = 2)
    # model = GradientBoostingClassifier(n_estimators=50,learning_rate=0.5, max_depth=6, random_state=0)
    model = RandomForestClassifier(criterion = 'entropy', random_state = 0)
    #model = DecisionTreeClassifier(max_depth=7,criterion = 'entropy', random_state = 0)
    #model = XGBClassifier(n_estimators=50,learning_rate=0.5,max_depth=3)
    #model = AdaBoostClassifier(random_state = 0)
    # model = AdaBoostClassifier(n_estimators = 300, learning_rate = 1,random_state = 0)
    #model.fit(X_train, y_train)
    model.fit(stackedX, inputy)
    return model

In [13]:
def stacked_prediction(members, model, inputX):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX)
    # make a prediction
    yhat = model.predict(stackedX)
    return yhat

In [14]:
def stacked_prediction_proba(members,model,inputX):
    # create dataset using ensemble
    stackedX = stacked_dataset(members,inputX)
    # make a prediction
    yhatProb = model.predict_proba(stackedX)
    return yhatProb

In [15]:
acc_train = []
acc_test = []
Precision_macro = []
Precision_micro = []
Recall_macro = []
Recall_micro = []
F1_macro = []
F1_micro = []
AUC_ovo = []
AUC_ovr = []
train_time = []
test_time = []

In [16]:
num_classes = 15

In [17]:
for i in range(0,5):

    X_train_cv, X_test_cv, y_train_cv, y_test_cv = train_test_split(x_smote, y_smote, test_size = 0.25,random_state = i)
    y_train_cv = y_train_cv - 1
    y_test_cv = y_test_cv - 1

    sc = StandardScaler()
    X_train_cv = sc.fit_transform(X_train_cv)
    X_test_cv = sc.transform(X_test_cv)
    unique_labels = np.unique(y_train_cv)
    print(unique_labels)

    start = time.time()
    # Model 1
    ann1 = Sequential()
    ann1.add(tf.keras.layers.Dense(units=15, activation='relu'))
    ann1.add(tf.keras.layers.Dense(units=20, activation='relu'))
    ann1.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
    ann1.compile(optimizer = 'adam', loss = kf.losses.SparseCategoricalCrossentropy(), metrics = [kf.metrics.CategoricalAccuracy()])
    ann1.fit(X_train_cv, y_train_cv, batch_size = 128, epochs = 100,verbose=0)

    # Model 2
    ann2 = Sequential()
    ann2.add(tf.keras.layers.Dense(units=10, activation='relu'))
    ann2.add(tf.keras.layers.Dense(units=15, activation='relu'))
    ann2.add(tf.keras.layers.Dense(units=25, activation='relu'))
    ann2.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
    ann2.compile(optimizer = 'adam', loss = kf.losses.SparseCategoricalCrossentropy(), metrics = [kf.metrics.CategoricalAccuracy()])
    ann2.fit(X_train_cv, y_train_cv, batch_size = 128, epochs = 150,verbose=0)

    # Model 3
    ann3 = Sequential()
    ann3.add(tf.keras.layers.Dense(units=20, activation='relu'))
    ann3.add(tf.keras.layers.Dense(units=25, activation='relu'))
    ann3.add(tf.keras.layers.Dense(units=30, activation='relu'))
    ann3.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
    ann3.compile(optimizer = 'adam', loss = kf.losses.SparseCategoricalCrossentropy(), metrics = [kf.metrics.CategoricalAccuracy()])
    ann3.fit(X_train_cv, y_train_cv, batch_size = 128, epochs = 180,verbose=0)

    # Model 4

    ann4 = Sequential()
    ann4.add(tf.keras.layers.Dense(units=30, activation='relu'))
    ann4.add(tf.keras.layers.Dense(units=35, activation='relu'))
    ann4.add(tf.keras.layers.Dense(units=40, activation='relu'))
    ann4.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
    ann4.compile(optimizer = 'adam', loss = kf.losses.SparseCategoricalCrossentropy(), metrics = [kf.metrics.CategoricalAccuracy()])
    ann4.fit(X_train_cv, y_train_cv, batch_size = 128, epochs = 200,verbose=0)

    # Model 5

    ann5 = Sequential()
    ann5.add(tf.keras.layers.Dense(units=40, activation='relu'))
    ann5.add(tf.keras.layers.Dense(units=50, activation='relu'))
    ann5.add(tf.keras.layers.Dense(units=55, activation='relu'))
    ann5.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
    ann5.compile(optimizer = 'adam', loss = kf.losses.SparseCategoricalCrossentropy(), metrics = [kf.metrics.CategoricalAccuracy()])
    ann5.fit(X_train_cv, y_train_cv, batch_size = 128, epochs = 250,verbose=0)

    # Model 6

    ann6 = Sequential()
    ann6.add(tf.keras.layers.Dense(units=35, activation='relu'))
    ann6.add(tf.keras.layers.Dense(units=45, activation='relu'))
    ann6.add(tf.keras.layers.Dense(units=50, activation='relu'))
    ann6.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
    ann6.compile(optimizer = 'adam', loss = kf.losses.SparseCategoricalCrossentropy(), metrics = [kf.metrics.CategoricalAccuracy()])
    ann6.fit(X_train_cv, y_train_cv, batch_size = 128, epochs = 300,verbose=0)

    # Model 7

    ann7 = Sequential()
    ann7.add(tf.keras.layers.Dense(units=35, activation='relu'))
    ann7.add(tf.keras.layers.Dense(units=45, activation='relu'))
    ann7.add(tf.keras.layers.Dense(units=50, activation='relu'))
    ann7.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
    ann7.compile(optimizer = 'adam', loss = kf.losses.SparseCategoricalCrossentropy(), metrics = [kf.metrics.CategoricalAccuracy()])
    ann7.fit(X_train_cv, y_train_cv, batch_size = 128, epochs = 300,verbose=0)

    # Model 8

    ann8 = Sequential()
    ann8.add(tf.keras.layers.Dense(units=35, activation='relu'))
    ann8.add(tf.keras.layers.Dense(units=45, activation='relu'))
    ann8.add(tf.keras.layers.Dense(units=50, activation='relu'))
    ann8.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
    ann8.compile(optimizer = 'adam', loss = kf.losses.SparseCategoricalCrossentropy(), metrics = [kf.metrics.CategoricalAccuracy()])
    ann8.fit(X_train_cv, y_train_cv, batch_size = 128, epochs = 300,verbose=0)

    # Model 9

    ann9 = Sequential()
    ann9.add(tf.keras.layers.Dense(units=35, activation='relu'))
    ann9.add(tf.keras.layers.Dense(units=45, activation='relu'))
    ann9.add(tf.keras.layers.Dense(units=50, activation='relu'))
    ann9.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
    ann9.compile(optimizer = 'adam', loss = kf.losses.SparseCategoricalCrossentropy(), metrics = [kf.metrics.CategoricalAccuracy()])
    ann9.fit(X_train_cv, y_train_cv, batch_size = 128, epochs = 300,verbose=0)

    # Model 10

    ann10 = Sequential()
    ann10.add(tf.keras.layers.Dense(units=35, activation='relu'))
    ann10.add(tf.keras.layers.Dense(units=45, activation='relu'))
    ann10.add(tf.keras.layers.Dense(units=50, activation='relu'))
    ann10.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
    ann10.compile(optimizer = 'adam', loss = kf.losses.SparseCategoricalCrossentropy(), metrics = [kf.metrics.CategoricalAccuracy()])
    ann10.fit(X_train_cv, y_train_cv, batch_size = 128, epochs = 300,verbose=0)
    # Model 11

    ann11 = Sequential()
    ann11.add(tf.keras.layers.Dense(units=35, activation='relu'))
    ann11.add(tf.keras.layers.Dense(units=45, activation='relu'))
    ann11.add(tf.keras.layers.Dense(units=50, activation='relu'))
    ann11.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
    ann11.compile(optimizer = 'adam', loss = kf.losses.SparseCategoricalCrossentropy(), metrics = [kf.metrics.CategoricalAccuracy()])
    ann11.fit(X_train_cv, y_train_cv, batch_size = 128, epochs = 300,verbose=0)

    # Model 12

    ann12 = Sequential()
    ann12.add(tf.keras.layers.Dense(units=35, activation='relu'))
    ann12.add(tf.keras.layers.Dense(units=45, activation='relu'))
    ann12.add(tf.keras.layers.Dense(units=50, activation='relu'))
    ann12.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
    ann12.compile(optimizer = 'adam', loss = kf.losses.SparseCategoricalCrossentropy(), metrics = [kf.metrics.CategoricalAccuracy()])
    ann12.fit(X_train_cv, y_train_cv, batch_size = 128, epochs = 300,verbose=0)

    # Model 13

    ann13 = Sequential()
    ann13.add(tf.keras.layers.Dense(units=35, activation='relu'))
    ann13.add(tf.keras.layers.Dense(units=45, activation='relu'))
    ann13.add(tf.keras.layers.Dense(units=50, activation='relu'))
    ann13.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
    ann13.compile(optimizer = 'adam', loss = kf.losses.SparseCategoricalCrossentropy(), metrics = [kf.metrics.CategoricalAccuracy()])
    ann13.fit(X_train_cv, y_train_cv, batch_size = 128, epochs = 300,verbose=0)

    # Model 14

    ann14 = Sequential()
    ann14.add(tf.keras.layers.Dense(units=35, activation='relu'))
    ann14.add(tf.keras.layers.Dense(units=45, activation='relu'))
    ann14.add(tf.keras.layers.Dense(units=50, activation='relu'))
    ann14.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
    ann14.compile(optimizer = 'adam', loss = kf.losses.SparseCategoricalCrossentropy(), metrics = [kf.metrics.CategoricalAccuracy()])
    ann14.fit(X_train_cv, y_train_cv, batch_size = 128, epochs = 300,verbose=0)

    # Model 15

    ann15 = Sequential()
    ann15.add(tf.keras.layers.Dense(units=35, activation='relu'))
    ann15.add(tf.keras.layers.Dense(units=45, activation='sigmoid'))
    ann15.add(tf.keras.layers.Dense(units=50, activation='relu'))
    ann15.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
    ann15.compile(optimizer = 'adam', loss = kf.losses.SparseCategoricalCrossentropy(), metrics = [kf.metrics.CategoricalAccuracy()])
    ann15.fit(X_train_cv, y_train_cv, batch_size = 128, epochs = 300,verbose=0)

    # Model 16

    ann16 = Sequential()
    ann16.add(tf.keras.layers.Dense(units=35, activation='relu'))
    ann16.add(tf.keras.layers.Dense(units=45, activation= 'sigmoid'))
    ann16.add(tf.keras.layers.Dense(units=50, activation='relu'))
    ann16.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
    ann16.compile(optimizer = 'adam', loss = kf.losses.SparseCategoricalCrossentropy(), metrics = [kf.metrics.CategoricalAccuracy()])
    ann16.fit(X_train_cv, y_train_cv, batch_size = 128, epochs = 300,verbose=0)

    ann1.save('north/model1.h5')
    ann2.save('north/model2.h5')
    ann3.save('north/model3.h5')
    ann4.save('north/model4.h5')
    ann5.save('north/model5.h5')
    ann6.save('north/model6.h5')
    ann7.save('north/model7.h5')
    ann8.save('north/model8.h5')
    ann9.save('north/model9.h5')
    ann10.save('north/model10.h5')
    ann11.save('north/model11.h5')
    ann12.save('north/model12.h5')
    ann13.save('north/model13.h5')
    ann14.save('north/model14.h5')
    ann15.save('north/model15.h5')
    ann16.save('north/model16.h5')

    members = load_all_models(16)
    model = fit_stacked_model(members, X_train_cv,y_train_cv)
    end = time.time()
    print("Cross validation Train time: ",(end-start) * 10**3, "ms")
    t = (end-start)*1000
    train_time.append(t)

    start = time.time()
    y_pred = stacked_prediction(members, model, X_test_cv)
    end = time.time()

    t = (end - start)*1000
    test_time.append(t)

    print("Cross validation Test time: ",(end-start) * 10**3, "ms")
    y_test_prob = stacked_prediction_proba(members,model,X_test_cv)
    y_train_pred = stacked_prediction(members, model, X_train_cv)

    cm = confusion_matrix(y_test_cv, y_pred)


    accuracy = accuracy_score(y_test_cv, y_pred)
    acc_test.append(accuracy)
    accuracy_train = accuracy_score(y_train_cv, y_train_pred)
    acc_train.append(accuracy_train)

    pre_macro = precision_score(y_test_cv, y_pred,average='macro')
    Precision_macro.append(pre_macro)
    pre_micro = precision_score(y_test_cv, y_pred,average='micro')
    Precision_micro.append(pre_micro)

    re_macro = recall_score(y_test_cv, y_pred,average='macro')
    Recall_macro.append(re_macro)
    re_micro = recall_score(y_test_cv, y_pred,average='micro')
    Recall_micro.append(re_micro)

    f1_macro = f1_score(y_test_cv, y_pred,average='macro')
    F1_macro.append(f1_macro)
    f1_micro = f1_score(y_test_cv, y_pred,average='micro')
    F1_micro.append(f1_micro)

    auc_ovo = roc_auc_score(y_test_cv,y_test_prob,multi_class='ovo')
    AUC_ovo.append(auc_ovo)
    auc_ovr = roc_auc_score(y_test_cv,y_test_prob,multi_class='ovr')
    AUC_ovr.append(auc_ovr)

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11.]


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Cross validation Train time:  1956981.5289974213 ms
Cross validation Test time:  9150.267601013184 ms
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11.]


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Cross validation Train time:  1952471.572637558 ms
Cross validation Test time:  8788.605451583862 ms
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11.]


...

Cross validation Train time:  1928654.6359062195 ms
Cross validation Test time:  7621.966123580933 ms
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11.]


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Cross validation Train time:  1993679.8586845398 ms
Cross validation Test time:  9397.96257019043 ms
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11.]


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Cross validation Train time:  1902297.9147434235 ms
Cross validation Test time:  8152.333736419678 ms


In [18]:
result_cv = pd.DataFrame()
result_cv["Train_acc"] = acc_train
result_cv["Test_acc"] = acc_test
result_cv["Precision(macro)"] = Precision_macro
result_cv["Precision(micro)"] = Precision_micro
result_cv["Recall(macro)"] = Recall_macro
result_cv["Recall(micro)"] = Recall_micro
result_cv["F1(macro)"] = F1_macro
result_cv["F1(micro)"] = F1_micro
result_cv["AUC(ovo)"] = AUC_ovo
result_cv["AUC(ovr)"] = AUC_ovr
result_cv["Train Time"] = train_time
result_cv["Test Time"] = test_time

In [19]:
train_time

[1956981.5289974213,
 1952471.572637558,
 1928654.6359062195,
 1993679.8586845398,
 1902297.9147434235]

In [20]:
test_time

[9150.267601013184,
 8788.605451583862,
 7621.966123580933,
 9397.96257019043,
 8152.333736419678]

In [21]:
result_cv.to_csv("filename", index=False)

In [22]:
result_cv_avg = pd.DataFrame()
lst = []
x = mean(acc_train)*100
lst.append(x)
result_cv_avg["Train_acc"] = lst
result_cv_avg["Test_acc"] = mean(acc_test)*100
result_cv_avg["SD"] = np.std(acc_test)*100
result_cv_avg["Precision(macro)"] = mean(Precision_macro)
result_cv_avg["Precision(micro)"] = mean(Precision_micro)
result_cv_avg["Recall(macro)"] = mean(Recall_macro)
result_cv_avg["Recall(micro)"] = mean(Recall_micro)
result_cv_avg["F1(macro)"] = mean(F1_macro)
result_cv_avg["F1(micro)"] = mean(F1_micro)
result_cv_avg["AUC(ovo)"] = mean(AUC_ovo)
result_cv_avg["AUC(ovr)"] = mean(AUC_ovr)

In [23]:
result_cv_avg.to_csv("filename", index=False)

In [24]:
acc_train

[0.9996510642540252,
 0.9994516723991825,
 0.9995015203628932,
 0.9994516723991825,
 0.9996510642540252]

In [25]:
acc_test

[0.9440705847166143,
 0.9392851802003889,
 0.9473605503215193,
 0.9445192163900105,
 0.9424256019141618]

In [26]:
Precision_macro

[0.9417013365775145,
 0.9379753656415334,
 0.9440656558193642,
 0.9440592979452189,
 0.9402224835967669]

In [27]:
Precision_micro

[0.9440705847166143,
 0.9392851802003889,
 0.9473605503215193,
 0.9445192163900105,
 0.9424256019141618]

In [28]:
Recall_macro

[0.9441334173319934,
 0.939741156256093,
 0.9458171059633393,
 0.9457043064301818,
 0.9415887006164173]

In [29]:
Recall_micro

[0.9440705847166143,
 0.9392851802003889,
 0.9473605503215193,
 0.9445192163900105,
 0.9424256019141618]

In [30]:
F1_macro

[0.9419048840943441,
 0.938173720696545,
 0.9445587130364341,
 0.944768863608299,
 0.940727761112712]

In [31]:
F1_micro

[0.9440705847166143,
 0.9392851802003889,
 0.9473605503215193,
 0.9445192163900106,
 0.9424256019141618]

In [32]:
AUC_ovo

[0.9960132905154334,
 0.9951626856551211,
 0.995796567400562,
 0.9958257885357333,
 0.9949923760562249]

In [33]:
index = np.argmax(AUC_ovo)
print(index)

0


In [34]:
AUC_ovr

[0.9960435435974028,
 0.9951665677680722,
 0.9959278185306001,
 0.9957476779829109,
 0.9950491421258567]

In [35]:
 print("Accuracy(Train): {:.2f} %".format(mean(acc_train)*100))
print("Accuracy(Test): {:.2f} %".format(mean(acc_test)*100))
print("Standard Deviation: {:.2f}%".format(np.std(acc_test)*100))
print("Precision (macro): %.3f" %mean(Precision_macro))
print("Precision (micro): %.3f" %mean(Precision_micro))
print("Recall (macro): %.3f" %mean(Recall_macro))
print("Recall (micro): %.3f" %mean(Recall_micro))
print("F1 (macro): %.3f" %mean(F1_macro))
print("F1 (micro): %.3f" %mean(F1_micro))
print("AUC (ovo): %.3f" %mean(AUC_ovo))
print("AUC (ovr): %.3f" %mean(AUC_ovr))

Accuracy(Train): 99.95 %
Accuracy(Test): 94.35 %
Standard Deviation: 0.27%
Precision (macro): 0.942
Precision (micro): 0.944
Recall (macro): 0.943
Recall (micro): 0.944
F1 (macro): 0.942
F1 (micro): 0.944
AUC (ovo): 0.996
AUC (ovr): 0.996


In [36]:
from numpy import interp

In [37]:
actual_classes = y_test_cv
predicted_proba = y_test_prob

fpr = dict()
tpr = dict()
roc_auc = dict()
n_classes=14
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(np.array(pd.get_dummies(actual_classes))[:, i], np.array(predicted_proba)[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
#     print("fpr: ",fpr[i])
#     print("tpr: ",tpr[i])
#     print("roc_auc: ",roc_auc[i])

all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
print("all_fpr: ",all_fpr)
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
print("mean_tpr: ",mean_tpr)
print("AUC = ",roc_auc['macro'])

IndexError: index 12 is out of bounds for axis 1 with size 12

In [ ]:
roc_csv = pd.DataFrame()
roc_csv["FPR(Macro)"] = all_fpr
roc_csv["TPR(Macro)"] = mean_tpr

In [ ]:
roc_csv.to_csv("filename", index=False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import numpy as np

# Assuming you have already calculated the confusion matrix 'cm' for 14 classes

# Normalize the confusion matrix
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sns.set(font_scale=1.2)
# Plot the normalized confusion matrix heatmap
fig=plt.figure(figsize=(11, 5))
sns.heatmap(cm_normalized, annot=True, fmt='.1f', cmap='Blues', xticklabels=range(14), yticklabels=range(14))
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
fig.savefig('RRR',dpi=300, bbox_inches = 'tight')
# plt.title('Normalized Confusion Matrix Heatmap')
plt.show()